# Medical Expenditure Panel Survey Data Scrape
### Because SAS never dies...###
----

The MEPS data set is fairly easy to obtain. All the ZIP files are stored at a consistent URL, making our lives fairly easy.

What's more challenging is actually making sense of the data. We ran into two issues:

- The data comes in the SAS XPORT format. R is the only language we could find actually capable of reading these files. In order to open the data, we had to turn them into CSV for everyone to use.
- Fixed-width formatting means there are plenty of values that make no sense: "-9" is actually a flag for "the interviewer did not record this data."

This notebook contains the code needed to scrape the MEPS site for turning data into CSV format.

In [1]:
library(httr)
library(foreign)

## Normalize PUF

In order to actually look up data, each file has a unique Public Use File (PUF) code associated with it. However, in order to download the actual file, the PUF has to be translated a bit.

- Remove the `C-` in all names - `HC-178F` becomes `h178f`
- Remove the leading 0 in all names - `HC-048E` becomes `h48e`
- Lowercase

In [2]:
normalize_puf <- function(full_puf) {
    stage_1 <- gsub("C-0", "", full_puf)
    stage_2 <- gsub("C-", "", stage_1)
    
    tolower(stage_2)
}

### Download PUF

The download process ended up being a bit easier than expected. All the files are located in a single directory, making the job of scraping trivial. Each **PUF** has two zip files associated with it:
- The ASCII fixed-width data
- The binary fixed-width data (in XPORT format)

The XPORT format contains information needed to reconstruct the column headers, so we use that to make our lives easy. However, we have to use a specfic library (foreign) to actually read the data. Finally, write everything to a CSV.

One point of improvement in the future would be to remove the intermediate files. For right now, we're not going to worry too much about them.

In [3]:
download_puf <- function(short_puf) {
    puf_base <- "http://meps.ahrq.gov/mepsweb/data_files/pufs/"
    puf_suffix <- "ssp.zip"
    
    zip_filename <- paste0(short_puf, "ssp.zip")
    filename <- paste0(short_puf, ".ssp")
    puf_url <- paste0(puf_base, zip_filename)
    download.file(puf_url, zip_filename)
    
    # unzip
    unzip(zip_filename, files = filename)
    saveName <- paste0(short_puf, ".csv")

    # read sas file and return as csv file
    mydata <- read.xport(filename)
    write.table(mydata, file = saveName, sep = ",")
}

### Actual Downloads
Now that the functions are defined, we can put them to use. Two example vectors have been provided to download all available data for emergency room visits, and for outpatient care over all available years.

In [4]:
# Example usage: Download data by PUF
#
# Emergency Visit PUF's
# pufs <- c('HC-160E','HC-152E','HC-144E','HC-135E','HC-126E','HC-118E','HC-110E','HC-102E','HC-094E','HC-085E','HC-077E','HC-067E','HC-059E','HC-051E','HC-033E','HC-026E','HC-016E','HC-010E')

# Outpatient Visits
# pufs <- c('HC-160F','HC-152F','HC-144F','HC-135F','HC-126F','HC-118F','HC-110F','HC-102F','HC-094F','HC-085F','HC-077F','HC-067F','HC-059F','HC-051F','HC-033F','HC-026F','HC-016F','HC-010F')

# puf_downloads = c()
# for (puf in pufs) {
#    download_puf(normalize_puf(puf))
# }

########

# Example usage: Scrape the MEPS web page for all PUF ids
# Feel free to modify the URL being used - Any of the links on [this page](http://meps.ahrq.gov/mepsweb/data_stats/download_data_files.jsp)
# are fair game.

#install.packages("rvest")
library(rvest)
html <-
  read_html(
    "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=2%2CHousehold+Event+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Emergency+Room+Visits"
  )

library(XML)

# to scrap all the file name from web and form html links
doc <-  htmlParse(html, asText = TRUE)
plain.text <- xpathSApply(doc, "//a", xmlValue)
data <- c()
for (i in 1:length(plain.text))
{
  if (substr(plain.text[i], 1,3) == "HC-")
  {
    data[i] <- tolower(plain.text[i])
  }
  originalData <- data[!is.na(data)]
  
}

mydata <- c()
for (i in 1:length(originalData))
{
  mydata[i] <- gsub("c-0", "", originalData[i])
  mydata[i] <- gsub("c-", "", mydata[i])
  
}

# to scrap all .ssp data file 
for (i in 1:length(mydata))
{
  download_puf(mydata[i])
}

ERROR: Error in eval(expr, envir, enclos): object 'w' not found


Loading required package: xml2

Attaching package: 'XML'

The following object is masked from 'package:rvest':

    xml

Warning message:
In is.na(data): is.na() applied to non-(list or vector) of type 'NULL'Warning message:
In is.na(data): is.na() applied to non-(list or vector) of type 'NULL'Warning message:
In is.na(data): is.na() applied to non-(list or vector) of type 'NULL'Warning message:
In is.na(data): is.na() applied to non-(list or vector) of type 'NULL'Warning message:
In is.na(data): is.na() applied to non-(list or vector) of type 'NULL'Warning message:
In is.na(data): is.na() applied to non-(list or vector) of type 'NULL'Warning message:
In is.na(data): is.na() applied to non-(list or vector) of type 'NULL'Warning message:
In is.na(data): is.na() applied to non-(list or vector) of type 'NULL'Warning message:
In is.na(data): is.na() applied to non-(list or vector) of type 'NULL'Warning message:
In is.na(data): is.na() applied to non-(list or vector) of type 'NULL'Warning m

ERROR: Error in lookup.xport(file): file not in SAS transfer format
